In [11]:
import importlib
import subprocess
import sys

def install_and_import(package, import_name=None):
    """
    Try to import a package. If it's not installed, install it via pip and import again.
    package: name on pip (e.g. 'pandas')
    import_name: name used in import (e.g. 'pandas' or 'matplotlib.pyplot').
                 If None, uses package.
    """
    if import_name is None:
        import_name = package

    try:
        return importlib.import_module(import_name)
    except ImportError:
        print(f"{import_name} not found, installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        return importlib.import_module(import_name)

# use it for your libs
pd = install_and_import("pandas")
np = install_and_import("numpy")
plt = install_and_import("matplotlib.pyplot", "matplotlib.pyplot")
# Use the function to install xgboost instead of direct pip command
xgb = install_and_import("xgboost")


In [34]:
import pandas as pd

filtered_df_anal = pd.read_csv(r"C:\Users\User\Downloads\filtered_data_anal.csv")

# első pár sor ellenőrzéshez:
print(filtered_df_anal.head())


   site_id_(telephely)  class_size  teachers_number_site  math_score_8_std  \
0              2829211          18                 28.25         -1.675153   
1              3353709           9                 13.00          0.265037   
2              3359201          13                 13.00         -1.494051   
3              3511901          23                 37.00          0.166714   
4              3108801          27                 37.50          1.045846   

   total_students_site mother_education_level father_education_level  \
0                  266         primary_school         primary_school   
1                   82                college  apprenticeship_school   
2                  108         primary_school         primary_school   
3                  343                college  secondary_with_matura   
4                  393  secondary_with_matura  apprenticeship_school   

  books_at_home  family_background_index_std student_gender  \
0          0_50                    

In [36]:
import pandas as pd                         # For data manipulation and handling DataFrames
from xgboost import XGBRegressor            # XGBoost regression model
from sklearn.model_selection import train_test_split, RandomizedSearchCV  # Train-test split + hyperparameter search
from sklearn.metrics import mean_squared_error, r2_score  # Regression evaluation metrics
import numpy as np                          # For numerical operations (e.g., square root)

target = "math_score_8_std"  # Name of the target variable we want to predict

# Feature matrix: drop target and ID columns
X = filtered_df_anal.drop(columns=[target])  # Feature matrix
y = filtered_df_anal[target]                           # Target vector: the column we want to predict

# Train–test split (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Base XGBoost model (without fixed hyperparameters)
base_model = XGBRegressor(
    objective="reg:squarederror",  # explicit regression objective
    tree_method="hist",            # efficient histogram-based algorithm
    random_state=42,
)

# Hyperparameter search space (coarse but reasonable for an assignment)
param_distributions = {
    "n_estimators": [100, 200, 300, 500, 700],
    "learning_rate": [0.01, 0.03, 0.05, 0.1, 0.15, 0.2],
    "max_depth": [3, 4, 5, 6, 7, 8, 10],
    "subsample": [0.6, 0.7, 0.8, 0.9, 1.0],
    "colsample_bytree": [0.6, 0.8, 1.0],
    "reg_lambda": [0, 1, 2, 5, 10],   # L2 regularization
    "reg_alpha": [0, 0.1, 0.25, 0.5, 0.75, 1], # L1 regularization
}

# Randomized hyperparameter search with cross-validation
random_search = RandomizedSearchCV(
    estimator=base_model,
    param_distributions=param_distributions,
    n_iter=30,                      # number of sampled hyperparameter combinations
    scoring="neg_root_mean_squared_error",  # we minimize RMSE
    cv=5,                           # 5-fold CV
    verbose=1,
    n_jobs=-1,                      # use all available cores
    random_state=42,
)

# Fit the hyperparameter search on the training data
random_search.fit(X_train, y_train)

# Best model after hyperparameter tuning
model = random_search.best_estimator_

print("Best hyperparameters:", random_search.best_params_)

# Use the tuned model to predict target values for the test set
y_pred = model.predict(X_test)

# Compute evaluation metrics for regression
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("RMSE:", rmse)
print("R^2:", r2)


Fitting 5 folds for each of 30 candidates, totalling 150 fits


ValueError: 
All the 150 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
150 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\User\anaconda3\Lib\site-packages\xgboost\data.py", line 407, in pandas_feature_info
    new_feature_types.append(_pandas_dtype_mapper[dtype.name])
                             ~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^
KeyError: 'object'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\User\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\User\anaconda3\Lib\site-packages\xgboost\core.py", line 774

In [37]:
import pandas as pd                         # Data manipulation
import numpy as np                          # Numerical operations
from xgboost import XGBRegressor            # XGBoost regression model
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score

# -----------------------------
# 1. Define target and features
# -----------------------------
target = "math_score_8_std"                 # Target variable to predict

# ID-like columns that should not be used as features (drop only if they exist)
id_cols = ["row_number", "student_id", "student_serial_number"]

# Build list of columns to drop safely (only those that exist in the DataFrame)
cols_to_drop = [target] + id_cols
cols_to_drop = [c for c in cols_to_drop if c in filtered_df_anal.columns]

# Feature matrix X and target vector y
X = filtered_df_anal.drop(columns=cols_to_drop)   # All predictors
y = filtered_df_anal[target]                      # Outcome

# ------------------------------------
# 2. Handle categorical (object) data
# ------------------------------------
# XGBoost needs numeric or pandas 'category' dtypes
cat_cols = X.select_dtypes(include=["object"]).columns
X[cat_cols] = X[cat_cols].astype("category")

# -----------------------------
# 3. Train–test split (80–20)
# -----------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# -----------------------------
# 4. Base XGBoost model
# -----------------------------
base_model = XGBRegressor(
    objective="reg:squarederror",  # Explicit regression objective
    tree_method="hist",            # Required for categorical support / efficiency
    enable_categorical=True,       # Tell XGBoost that we use categorical features
    random_state=42,
)

# --------------------------------------------
# 5. Hyperparameter search space (Randomized)
# --------------------------------------------
param_distributions = {
    "n_estimators": [100, 200, 300, 500, 700],
    "learning_rate": [0.01, 0.03, 0.05, 0.1, 0.15, 0.2],
    "max_depth": [3, 4, 5, 6, 7, 8, 10],
    "subsample": [0.6, 0.7, 0.8, 0.9, 1.0],
    "colsample_bytree": [0.6, 0.8, 1.0],
    "reg_lambda": [0, 1, 2, 5, 10],             # L2 regularization
    "reg_alpha": [0, 0.1, 0.25, 0.5, 0.75, 1],  # L1 regularization
}

random_search = RandomizedSearchCV(
    estimator=base_model,
    param_distributions=param_distributions,
    n_iter=30,                         # Number of sampled hyperparameter combinations
    scoring="neg_root_mean_squared_error",  # Minimize RMSE
    cv=5,                              # 5-fold cross-validation
    verbose=1,
    n_jobs=-1,                         # Use all available cores
    random_state=42,
)

# ---------------------------------------------
# 6. Fit hyperparameter search & evaluate model
# ---------------------------------------------
random_search.fit(X_train, y_train)

# Best model after tuning
model = random_search.best_estimator_

print("Best hyperparameters:", random_search.best_params_)

# Predictions on test set
y_pred = model.predict(X_test)

# Regression metrics
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("RMSE:", rmse)
print("R^2:", r2)


Fitting 5 folds for each of 30 candidates, totalling 150 fits
Best hyperparameters: {'subsample': 0.8, 'reg_lambda': 1, 'reg_alpha': 0.5, 'n_estimators': 700, 'max_depth': 7, 'learning_rate': 0.03, 'colsample_bytree': 1.0}
RMSE: 0.8017034061515965
R^2: 0.31020580777827944


In [38]:
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from xgboost import plot_importance

# -------------------------------------------------------------------
# Global plotting style for publication-quality figures
# -------------------------------------------------------------------
mpl.rcParams.update({
    "font.family": "Times New Roman",
    "font.size": 11,
    "axes.titlesize": 13,
    "axes.labelsize": 12,
    "xtick.labelsize": 10,
    "ytick.labelsize": 10,
    "axes.spines.top": False,
    "axes.spines.right": False,
    "figure.dpi": 600,
})

save_dir = r"C:\Users\User\Desktop\Figures\Figures"
os.makedirs(save_dir, exist_ok=True)

# -------------------------------------------------------------------
# 1. True vs. predicted values
# -------------------------------------------------------------------
fig, ax = plt.subplots(figsize=(6, 6))

ax.scatter(y_test, y_pred, s=20, alpha=0.6)
ax.set_xlabel("True $math\\_score\\_8\\_std$")
ax.set_ylabel("Predicted $math\\_score\\_8\\_std$")
ax.set_title("True vs. predicted values")

min_val = min(y_test.min(), y_pred.min())
max_val = max(y_test.max(), y_pred.max())
ax.plot([min_val, max_val], [min_val, max_val],
        linestyle="--", linewidth=1)

ax.set_xlim(min_val, max_val)
ax.set_ylim(min_val, max_val)

ax.grid(True, linestyle=":", linewidth=0.5, alpha=0.7)

fig.tight_layout()
fig.savefig(os.path.join(save_dir, "true_vs_predicted.png"), dpi=600)
plt.close(fig)

# -------------------------------------------------------------------
# 2. Residuals histogram
# -------------------------------------------------------------------
residuals = y_test - y_pred

fig, ax = plt.subplots(figsize=(6, 4))

ax.hist(residuals, bins=30, edgecolor="black", alpha=0.8)
ax.set_xlabel("Residual (true – predicted)")
ax.set_ylabel("Frequency")
ax.set_title("Distribution of residuals")

ax.axvline(0, linestyle="--", linewidth=1)

ax.grid(True, linestyle=":", linewidth=0.5, alpha=0.7)

fig.tight_layout()
fig.savefig(os.path.join(save_dir, "residuals_histogram.png"), dpi=600)
plt.close(fig)

# -------------------------------------------------------------------
# 3. Residuals vs. fitted values
# -------------------------------------------------------------------
fig, ax = plt.subplots(figsize=(6, 4))

ax.scatter(y_pred, residuals, s=20, alpha=0.6)
ax.axhline(0, linestyle="--", linewidth=1)

ax.set_xlabel("Predicted $math\\_score\\_8\\_std$")
ax.set_ylabel("Residual (true – predicted)")
ax.set_title("Residuals vs. predicted values")

ax.grid(True, linestyle=":", linewidth=0.5, alpha=0.7)

fig.tight_layout()
fig.savefig(os.path.join(save_dir, "residuals_vs_fitted.png"), dpi=600)
plt.close(fig)

# -------------------------------------------------------------------
# 4. Feature importance (XGBoost)
# -------------------------------------------------------------------
fig, ax = plt.subplots(figsize=(7, 9))

plot_importance(
    model,
    max_num_features=20,
    ax=ax
)

ax.set_title("Top 20 feature importances (XGBoost)")

fig.tight_layout()
fig.savefig(os.path.join(save_dir, "feature_importance.png"), dpi=600)
plt.close(fig)
